# 9.8.2 Income Redistribution Model

Agent p's reward is, $u_p(s_p - x_p) = \frac{(s_p - x_p)^{1-\alpha_p}}{1-\alpha_p}$
,where $s_p$ is a predetermined level of wealth and $x_p$ is the amount of investment.

Wealth evolution is, $h_p(x_{p, t}, \epsilon_{p,t+1}) = \gamma_p x_{p,t} + \epsilon_{p,t+1} x_{p,t}^{\beta_p}$

Then the realized wealth for agent p at time t+1 is, $s_{p, t+1} = (1 - \psi) h_p(x_{p, t}, \epsilon_{p,t+1}) + \psi h_q(x_{q, t}, \epsilon_{q,t+1})$
,where $\psi$ is the pre-determined wealth share rate between the players.

Consider two players case, each denoted by 1 and 2, who live forever. 

The time independent state transition function is defined as follows.

$g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2) = ((1 - \psi) h_1(x_1, \epsilon_1)+ \psi h_2(x_2, \epsilon_2),\ (1 - \psi) h_2(x_2, \epsilon_2)+ \psi h_1(x_1, \epsilon_1))$

Using the above functions, and defining $V_p$ as the value functions, we can set the system of Bellman equations as follows.

$\begin{cases} V_1(s_1, s_2) = \max_{0\leq x_1\leq s_1} \left\{ u_1(s_1 - x_1) + \delta E_{\epsilon}\left[ V_1(g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2)) \right] \right\}\\[10pt]
V_2(s_1, s_2) = \max_{0\leq x_2\leq s_2} \left\{ u_2(s_2 - x_2) + \delta E_{\epsilon}\left[ V_2(g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2)) \right] \right\}
\end{cases}$

I solve the above systems of functional equations by using collocation method, which means searching the Markov perfect equilibrium in this model.

https://github.com/QuantEcon/CompEcon.jl/blob/master/src/core.jl

ここに書いてあるように、QuantEconのfunevalでorderを指定すると、中でbasisMatricesのorderを指定してsplineの微分による評価を得られるよ。

In [1]:
using BasisMatrices
using QuantEcon

In [14]:
n = 5
smin = 3
smax = 11
maxit = 200
num_nodes = 3;

In [17]:
# make collocation matrix
sgrid0 = linspace(smin, smax, n)
basis = Basis(SplineParams(sgrid0, 0, 3), SplineParams(sgrid0, 0, 3))
S, (coordx, coordy) = nodes(basis)
Φ = BasisMatrix(basis, Expanded(), S, 0)

BasisMatrix{BasisMatrices.Expanded} of order [0 0]

In [18]:
# alpha, beta, gamma, psi are model's parameters
mutable struct IRG
    alpha::Array{Float64,1}
    beta::Array{Float64,1}
    gamma::Array{Float64,1}
    sigma::Array{Float64, 1}
    psi::Float64
    delta::Float64
end

# functions in this model
# vector valued functions
function u(model::IRG, s::Array{Float64, 2}, x::Array{Float64, 2}, p::Int64)
    return ((s[:, p] - x[:, p]).^(1 - model.alpha[p]))./(1 - model.alpha[p])
end

function h(model::IRG, x::Array{Float64, 2}, epsilon::Float64, p::Int64)
    return model.gamma[p] * x[:, p] + epsilon * (x[:, p].^(model.beta[p]))
end

function s(model::IRG, x::Array{Float64, 2}, epsilon::Array{Float64, 1})
    box = zeros((size(x)[1], 2))
    box[:, 1] = (1 - model.psi)*h(model, x, epsilon[1], 1) + model.psi*h(model, x, epsilon[2], 2)
    box[:, 2] = (1 - model.psi)*h(model, x, epsilon[2], 2) + model.psi*h(model, x, epsilon[1], 1)
    return box
end

# derivatives of  payoff, transition functions
# utility function
function ux(model::IRG, s::Array{Float64, 2}, x::Array{Float64, 2}, p::Int64)
    return -(s[:, p] - x[:, p]).^(-model.alpha[p])
end

function uxx(model::IRG, s::Array{Float64, 2}, x::Array{Float64, 2}, p::Int64)
    return -model.alpha[p]*(s[:, p] - x[:, p]).^(-model.alpha[p]-1)
end

# transition function
function sx(model::IRG, x::Array{Float64, 2}, epsilon::Array{Float64, 1},  p::Int64)
    return (1 - model.psi)*(model.gamma[p] + epsilon[p]*model.beta[p]*x[:, p].^(model.beta[p] - 1))
end

function sxx(model::IRG, x::Array{Float64, 2}, epsilon::Array{Float64, 1},  p::Int64)
    return (1 - model.psi)*epsilon[p]*model.beta[p]*(model.beta[p] - 1)*x[:, p].^(model.beta[p] - 2)
end

sxx (generic function with 1 method)

In [33]:
# Gaussian Quadrature
# using QuantEcon package for numerical integration w.r.t the expectation
# 3 nodes Gaussian quadrature of standard lognormal distribution
epss, weights = qnwlogn([num_nodes, num_nodes], [0,0], diagm([0.01, 0.01]));

# collocation function
function vmax(model::IRG, colnodes::Array{Float64, 2}, action::Array{Float64, 2}, coef::Array{Float64, 2})
    xnew = action
    v = zeros((size(colnodes)[1], 2))
    for p in 1:2
        xl, xu = 0.0, colnodes[:, p]
        order1 = [0 0]
        order1[1, p] = 1
        order2 = [0 0]
        order2[1, p] = 2
        for it in 1:maxit
            util, util_der1, util_der2 = u(model, colnodes, action, p), ux(model, colnodes, action, p), uxx(model, colnodes, action, p) 
            # println(util[1])
            # println(util_der1[1])
            # println(util_der2[1])
            # println(action)
            Ev, Evx, Evxx = 0.0, 0.0, 0.0
            for k in 1:num_nodes^2
                println(k)
                eps, weight= epss[k, :], weights[k]
                transition, transition_der1, transition_der2 = s(model, action, eps), sx(model, action, eps, p), sxx(model, action, eps, p)
                #println(transition)
                #println(transition_der1)
                #println(transition_der2)
                # coefによって結果が変わらないんだけど、係数はここにしか出てないからこいつがやっぱり怪しいよな
                # 係数は別に良いっぽいダメなのはfuneval
                println(coef[:, p])
                vn = funeval(coef[:, p], basis, transition)
                vnder1 =  funeval(coef[:, p], basis, transition, order1)
                vnder2 = funeval(coef[:, p], basis, transition, order2)
                # vnが係数と一致してるんだけどなにこれ
                println(vn)
                println(vnder1)
                #println(vnder2)
                Ev += weight * vn
                Evx += weight* vnder1.* transition_der1
                Evxx += weight * (vnder1.*transition_der2 + vnder2 .* (transition_der1.^2))
            end
            # println(Ev[1])
            # println(Evx[1])
            # println(Evxx[1])
            v[:, p] = util + Ev
            delx = -(util_der1 + model.delta * Evx) ./ (util_der2 + model.delta*Evxx)
            # delxがでかすぎる
            # println(delx)
            # println(xl-action[:, p])
            # println(xu-action[:, p])
            delx = min.(max.(delx, xl-action[:, p]), xu-action[:, p])
            action[:, p] = action[:, p] + delx
            if norm(delx) < tol
                break
            end
        end
        xnew[:, p] = action[:, p]
    end
    return v, xnew
end

vmax (generic function with 1 method)

In [36]:
# 係数の初期値を変えても結果に変化がない。これは変じゃね？
# 係数の初期値を乱数にしてみると何だかうまくいってるように見えるので、そういう感じでやる。
initial = rand((size(Φ.vals[1])[1]), 2)
x = zeros((size(Φ.vals[1])[1]), 2)+1
model = IRG([0.2,0.2], [0.5, 0.5], [0.9,0.9], [0.1,0.1], 0.05, 0.9)
tol = 0.01

vmax(model, S, x, initial)

1
[0.745422, 0.459093, 0.62584, 0.0178684, 0.464023, 0.11874, 0.409644, 0.134828, 0.765732, 0.0116928, 0.138271, 0.39447, 0.86733, 0.145446, 0.0693093, 0.530601, 0.29839, 0.484055, 0.442285, 0.535451, 0.5138, 0.21036, 0.644995, 0.990158, 0.908191, 0.218683, 0.637953, 0.693392, 0.135142, 0.200945, 0.205164, 0.332327, 0.0971263, 0.534067, 0.555836, 0.467409, 0.190433, 0.176934, 0.462397, 0.358681, 0.399453, 0.130215, 0.848445, 0.457646, 0.916749, 0.118677, 0.256981, 0.461497, 0.875418, 0.56183, 0.969854, 0.756654, 0.860831, 0.343383, 0.772545, 0.543746, 0.68237, 0.2398, 0.569682, 0.653609, 0.801279, 0.587541, 0.946778, 0.244788]
[-27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.6174; -27.

LoadError: [91mBoundsError: attempt to access 13-element Array{Float64,1} at index [0][39m

solve

In [22]:
# solving
initial = zeros((size(Φ.vals[1])[1]), 2)+1
x = zeros((size(Φ.vals[1])[1]), 2)+2
model = IRG([0.2,0.2], [0.5, 0.5], [0.9,0.9], [0.1,0.1], 0.05, 0.9)
tol = 0.1
# iteration for coefficients
c = initial
for it in 1:maxit
    cold = c
    v, x = vmax(model, S, x, c)
    c = Φ.vals[1] \ v
    if norm(cold - c) < tol
        break
    end
end

[1.0, 1.0][4.00225, 4.00225]

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] max(::Array{Float64,2}, ::Array{Float64,1}) at ./deprecated.jl:57
 [3] vmax(::IRG, ::Array{Float64,2}, ::Array{Float64,2}, ::Array{Float64,2}) at ./In[7]:30
 [4] macro expansion at ./In[22]:11 [inlined]
 [5] anonymous at ./<missing>:?
 [6] include_string(::String, ::String) at ./loading.jl:515
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/susu/.julia/v0.6/IJulia/src/execute_request.jl:160
 [8] eventloop(::ZMQ.Socket) at /Users/susu/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##11#14)() at ./task.jl:335
while loading In[22], in expression starting on line 8


[NaN, NaN][NaN, NaN][NaN, NaN][NaN, NaN][NaN, NaN][NaN, NaN]

LoadError: [91mInterruptException:[39m